In [11]:
import pandas as pd
import numpy as np
import os
from scipy.stats import zscore

# Path to folder containing raw CSV files
data_folder = r'C:\Users\jahna\OneDrive\Documents\TIME_SERIES_STOCK_FORECASTING\Data\nifty50-stock-market-data'

# Output folder for processed files
output_folder =  r'C:\Users\jahna\OneDrive\Documents\TIME_SERIES_STOCK_FORECASTING\PREPROCESSING\CLEAN_DATA_FINAL'
os.makedirs(output_folder, exist_ok=True)

# ------------------------------
# Utility Functions
# ------------------------------

def clean_column_names(df):
    """Standardizes column names to lowercase with snake_case."""
    df.columns = [col.strip().lower().replace(" ", "_").replace("%", "percent").replace("-", "_") for col in df.columns]
    return df

def remove_outliers(df, columns, z_thresh=4):
    """Removes outliers from numeric columns based on Z-score."""
    for col in columns:
        if col in df.columns:
            df = df[(np.abs(zscore(df[col].fillna(0))) < z_thresh)]
    return df

# Main Preprocessing Function


def preprocess_stock_data(df):
    # 1. Standardize column names
    df = clean_column_names(df)

    # 2. Convert 'date' column to datetime

    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df.dropna(subset=['date'], inplace=True)
    df.sort_values('date', inplace=True)
    df.set_index('date', inplace=True)

    # 3. Drop duplicate rows
    df.drop_duplicates(inplace=True)

    # 4. Drop columns with >80% missing values
    threshold = 0.8 * len(df)
    df.dropna(thresh=threshold, axis=1, inplace=True)

    # 5. Convert applicable columns to numeric
    for col in df.columns:
        if col not in ['symbol', 'series']:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # 6. Impute missing numeric values with median
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

    # 7. Sanity checks: remove rows with invalid or zero values
    if 'volume' in df.columns:
        df = df[df['volume'] > 0]
    if 'turnover' in df.columns:
        df = df[df['turnover'] > 0]

    # 8. Remove outliers using Z-score
    df = remove_outliers(df, ['close', 'volume', 'turnover'])

    # 9. Add date/time-based features
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['day'] = df.index.day
    df['weekday'] = df.index.dayofweek
    df['is_month_start'] = df.index.is_month_start
    df['is_month_end'] = df.index.is_month_end

    # 10. Rolling statistics
    df['rolling_5_close'] = df['close'].rolling(5).mean()
    df['rolling_20_close'] = df['close'].rolling(20).mean()
    df['rolling_5_volume'] = df['volume'].rolling(5).mean()
    df['rolling_std_10'] = df['close'].rolling(10).std()

    # 11. Lag features
    df['lag_1_close'] = df['close'].shift(1)
    df['lag_3_close'] = df['close'].shift(3)

    # Reset index to make 'date' a column again
    return df.reset_index()

# ------------------------------
# Loop Over All CSV Files
# ------------------------------
for file in os.listdir(data_folder):
    if file.endswith(".csv"):
        input_path = os.path.join(data_folder, file)
        try:
            df = pd.read_csv(input_path)

            # --- Step 1: Check if a date column exists ---
            # Case insensitive check
            date_cols = [col for col in df.columns if col.lower() == 'date']
            if not date_cols:
                raise ValueError("Missing 'Date' column.")

            # Rename to standard 'date'
            df.rename(columns={date_cols[0]: 'date'}, inplace=True)

            # --- Step 2: Try converting to datetime and check for valid entries ---
            df['date'] = pd.to_datetime(df['date'], errors='coerce')
            if df['date'].isna().all():
                raise ValueError("'Date' column has no valid datetime entries.")

            # Proceed with full preprocessing
            cleaned_df = preprocess_stock_data(df)
            output_path = os.path.join(output_folder, file)
            cleaned_df.to_csv(output_path, index=False)
            print(f"✅ Processed: {file}")

        except Exception as e:
            print(f"❌ Skipped: {file} | Reason: {e}")





✅ Processed: ADANIPORTS.csv
✅ Processed: ASIANPAINT.csv
✅ Processed: AXISBANK.csv
✅ Processed: BAJAJ-AUTO.csv
✅ Processed: BAJAJFINSV.csv
✅ Processed: BAJFINANCE.csv
✅ Processed: BHARTIARTL.csv
✅ Processed: BPCL.csv
✅ Processed: BRITANNIA.csv
✅ Processed: CIPLA.csv
✅ Processed: COALINDIA.csv
✅ Processed: DRREDDY.csv
✅ Processed: EICHERMOT.csv
✅ Processed: GAIL.csv
✅ Processed: GRASIM.csv
✅ Processed: HCLTECH.csv
✅ Processed: HDFC.csv
✅ Processed: HDFCBANK.csv
✅ Processed: HEROMOTOCO.csv
✅ Processed: HINDALCO.csv
✅ Processed: HINDUNILVR.csv
✅ Processed: ICICIBANK.csv
✅ Processed: INDUSINDBK.csv
✅ Processed: INFY.csv
✅ Processed: IOC.csv
✅ Processed: ITC.csv
✅ Processed: JSWSTEEL.csv
✅ Processed: KOTAKBANK.csv
✅ Processed: LT.csv
✅ Processed: MARUTI.csv
✅ Processed: MM.csv
✅ Processed: NESTLEIND.csv
✅ Processed: NIFTY50_all.csv
✅ Processed: NTPC.csv
✅ Processed: ONGC.csv
✅ Processed: POWERGRID.csv
✅ Processed: RELIANCE.csv
✅ Processed: SBIN.csv
✅ Processed: SHREECEM.csv
✅ Processed: SUNP